# IMC-DIMR and marker-individual exports

Hot pixel removal by removing local maxima in adjacent 3x3 sliding windows and export by marker

In [2]:
import sys
sys.path.append("/home/daria_l/src/MapMet/code/python/Image_Preprocessing/IMC-Denoise_adapted")

In [3]:
import IMC_Denoise
from IMC_Denoise.IMC_Denoise_main.DIMR import DIMR
import pandas as pd
import os, glob
import tifffile

After spillover compensation in R, we first have to convert the images to float32 as they are written from R in uint32. And we also have to attach channel names to the ome-tiffs.

In [ ]:
import os
import tifffile
import pandas as pd
import glob
from xtiff import to_tiff
import numpy as np

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"
samples = os.listdir(os.path.join(base_path, "BM"))

out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik")
marker_codes = pd.read_csv(os.path.join(out_path, 'marker_codes.csv'), sep=";")

for s in samples:
    mc_image_files = os.listdir(os.path.join(base_path, "BM", s, "IF_IMC_comp"))

    for f in mc_image_files:
        img = tifffile.imread(os.path.join(base_path, "BM", s, "IF_IMC_comp", f))
        img= img.astype(np.float32)

        to_tiff(
            img,
            os.path.join(base_path, "BM", s, "IF_IMC_comp", f),
            image_name=f,
            channel_names=list(marker_codes["new_names"]),
            pixel_size=1.0,
            pixel_depth=1.0,
            )

After spillover compensation, we have to prepare the file structure that is required by IMC-Denoise for DIMR hot-pixel removal.

In [ ]:
import os
import tifffile
import xml.etree.ElementTree
import pandas as pd
import io
import random
import glob
from xtiff import to_tiff
import numpy as np

def read_channel_names(path):
    
    tiff = tifffile.TiffFile(path)
    omexml_string = tiff.pages[0].description
    root = xml.etree.ElementTree.parse(io.StringIO(omexml_string))
    namespaces = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
    channels = root.findall('ome:Image[1]/ome:Pixels/ome:Channel', namespaces)
    channel_names = [c.attrib['Name'] for c in channels]
    return channel_names


base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria"

BM_path = os.path.join(base_path, "BM")
out_path = os.path.join(base_path, "_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik")

if not os.path.exists(os.path.join(out_path, "training")):
    os.mkdir(os.path.join(out_path, "training"))

if not os.path.exists(os.path.join(out_path, "other")):
    os.mkdir(os.path.join(out_path, "other"))

samples = os.listdir(BM_path)

for s in samples: 

    multichannel_path = os.path.join(BM_path, s, "IF_IMC_comp")

    mc_image_files = os.listdir(multichannel_path)

    training_file = random.choice(mc_image_files)
    training_file = training_file.split(".")[0]


    for f in mc_image_files:
        file_name = f.split(".")[0]

        mc_image = tifffile.imread(os.path.join(multichannel_path, f))

        channel_names = read_channel_names(os.path.join(multichannel_path, f))

        if file_name == training_file:
            folder = "training"
        else:
            folder = "other"

        if not os.path.exists(os.path.join(out_path, folder, "comp", file_name)):
            os.mkdir(os.path.join(out_path, folder, "comp", file_name))

        for c, img in enumerate(mc_image):
            tifffile.imwrite(os.path.join(out_path, folder, "comp", file_name, channel_names[c]+".tiff"),img)


After we have generated single images out of the stacks for each RoI of each sample, we can apply DIMR hot-pixel removal. We will randomly select one RoI per sample for training of DIMR and Ilastik background prediction. 

In [4]:
base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik"


marker_codes = pd.read_csv(os.path.join(base_path, 'marker_codes.csv'), sep=";")

n_neighbours = 4 # Larger n enables removing more consecutive hot pixels. 
n_iter = 3 # Iteration number for DIMR
window_size = 3 # Slide window size. For IMC images, window_size = 3 is fine.

other_path = os.path.join(base_path, "other")
training_path = os.path.join(base_path, "training")

for m in marker_codes["new_names"]:

    if m in ["131Xe-crtl1_Xe131", "134Xe-crtl2_Xe134", "136Ba-crtl3_Ba136"]:
        continue

    working_path = training_path

    if not os.path.exists(os.path.join(working_path, "DIMR", m)):
        os.mkdir(os.path.join(working_path, "DIMR", m))

    # if not os.path.exists(os.path.join(working_path, "DIMR_Ilastik", m)):
    #     os.mkdir(os.path.join(working_path, "DIMR_Ilastik", m))

    for s in glob.glob(os.path.join(working_path + "/comp/**/*{}.tiff".format(m)), recursive=True):
        img_raw = tifffile.imread(s)
        img_DIMR = DIMR(n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size).perform_DIMR(img_raw)
        tifffile.imwrite(os.path.join(working_path, "DIMR", m, s.split("/")[-2].split("_IF")[0] + "_" + s.split("/")[-1]), img_DIMR)


    working_path = other_path

    if not os.path.exists(os.path.join(working_path, "DIMR", m)):
        os.mkdir(os.path.join(working_path, "DIMR", m))

    # if not os.path.exists(os.path.join(working_path, "DIMR_Ilastik", m)):
    #     os.mkdir(os.path.join(working_path, "DIMR_Ilastik", m))

    for s in glob.glob(os.path.join(working_path + "/comp/**/*{}.tiff".format(m)), recursive=True):
        img_raw = tifffile.imread(s)
        img_DIMR = DIMR(n_neighbours = n_neighbours, n_iter = n_iter, window_size = window_size).perform_DIMR(img_raw)
        tifffile.imwrite(os.path.join(working_path, "DIMR", m, s.split("/")[-2].split("_IF")[0] + "_" + s.split("/")[-1]), img_DIMR)
    

After DIMR correction, we will perform background prediciton in Ilastik. 

In [1]:
import numpy as np
import tifffile
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

base_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20230112_DIMR_Ilastik"
subfolder = "training"

# markers = ["89Y-MPO_Y89", "115In-CD44_In115", "139La-Fibronectin_La139", "142Nd-CD11b_Nd142", "143Nd-HLA-DR_Nd143", "144Nd-PRPH_Nd144", "145Nd-CXCR2_Nd145", "146Nd-H3K9Ac_Nd146", 
# "147Sm-HLA-ABC_Sm147", "148Nd-CD20_Nd148", "149Sm-LUM_Sm149", "150Nd-CD11c_Nd150", "151Eu-CD24_Eu151", "152Sm-CD3_Sm152", "153Eu-CD45_Eu153", "154Sm-CD8a_Sm154", "155Gd-GD2_Gd155", 
# "156Gd-CD34_Gd156", "158Gd-CD10_Gd158", "159Tb-CXCR4_Tb159", "161Dy-S100B_Dy161"]

markers = ["165Ho-CD279_Ho165", "166Er-PNMT_Er166", "167Er-H4K12Ac_Er167", "168Er-GATA3_Er168"]

if not os.path.exists(os.path.join(base_path, subfolder, "processed")):
    os.mkdir(os.path.join(base_path, subfolder, "processed"))

for marker in markers: 
    samples = os.listdir(os.path.join(base_path, subfolder, "DIMR", marker))

    if not os.path.exists(os.path.join(base_path, subfolder, "processed", marker)):
        os.mkdir(os.path.join(base_path, subfolder, "processed", marker))

    for s in samples:
        sample = s.split(".")[0]

        img_DIMR = tifffile.imread(os.path.join(base_path, subfolder, "DIMR", marker, sample + ".tiff"))
        img_DIMR_99 = img_DIMR.copy()
        img_DIMR_99[img_DIMR_99 > np.percentile(img_DIMR_99, 99.9)] = np.percentile(img_DIMR_99, 99.9)
        #tifffile.imwrite(os.path.join(base_path, subfolder, "processed", marker, sample + "_dimr.tiff"), img_DIMR)
        #tifffile.imwrite(os.path.join(base_path, subfolder, "test/" + sample + "_dimr_99.tiff"), img_DIMR_99)

        mask_ilastik = tifffile.imread(os.path.join(base_path, subfolder, "DIMR_Ilastik", marker, sample + "_Simple Segmentation.tiff"))

        mask_ilastik[mask_ilastik==2] = 0
        #img_DIMR_corrected = img_DIMR * mask_ilastik
        #tifffile.imwrite(os.path.join(base_path, subfolder, "processed", marker, sample + "_DIMR_corrected.tiff"), img_DIMR_corrected)

        img_DIMR_99_corrected = img_DIMR_99 * mask_ilastik
        #tifffile.imwrite(os.path.join(base_path, subfolder, "processed", marker, sample + "_DIMR_99_corrected.tiff"), img_DIMR_99_corrected)

        scaler = MinMaxScaler()
        temp = img_DIMR_99_corrected.reshape(-1, 1)
        temp_norm = scaler.fit_transform(temp)
        img_DIMR_99_corrected_norm = temp_norm.reshape(img_DIMR_99_corrected.shape)
        tifffile.imwrite(os.path.join(base_path, subfolder, "processed", marker, sample + "_DIMR_99_corrected_norm.tiff"), img_DIMR_99_corrected_norm)